# Huawei Inverter Direct Connection

This notebook demonstrates how to directly connect to a Huawei Inverter using the `huawei-solar` library.

## Requirements

Ensure you have Python >= 3.11 installed.

Install the required library:
```sh
pip install huawei-solar==2.4.0b5
```


In [ ]:
!pip install huawei-solar==2.4.0b5

## Important Note

Huawei Inverter only allows 1 client at the monitoring port. If you are not using HomeAssistant, you can use this script to connect directly to the inverter.


In [ ]:
import asyncio
import logging
from huawei_solar import create_tcp_bridge, register_names as rn, HuaweiSolarException
#pip install huawei-solar==2.4.0b5

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger("huawei_solar_reader")

async def test_connection(host, port, slave_id):
    """Test the basic connection with the inverter"""
    try:
        logger.info(f"Trying to connect to {host}:{port} with slave_id={slave_id}")
        bridge = await create_tcp_bridge(
            host=host,
            port=port,
            slave_id=slave_id
        )
        
        logger.info("Connection established, getting basic information...")
        
        # Try to read a single register first to verify communication
        try:
            device_status = await bridge.client.get(rn.DEVICE_STATUS, slave_id)
            logger.info(f"Successful read - Device status: {device_status.value}")
            
            logger.info(f"Inverter model: {bridge.model_name}")
            logger.info(f"Serial number: {bridge.serial_number}")
            logger.info(f"Power meter type: {bridge.power_meter_type}")
            
            return bridge
            
        except Exception as e:
            logger.error(f"Error reading registers: {str(e)}")
            await bridge.stop()
            return None
            
    except Exception as e:
        logger.error(f"Error establishing connection: {str(e)}")
        return None

async def main():
    # Connection configuration
    host = "192.168.0.23"
    port = 6607
    
    # Test with different common slave_ids
    slave_ids = [0, 1]  # The most common are 0 and 1
    
    bridge = None
    for slave_id in slave_ids:
        logger.info(f"\nTesting with slave_id={slave_id}")
        bridge = await test_connection(host, port, slave_id)
        if bridge:
            logger.info(f"Successful connection with slave_id={slave_id}")
            break
    
    if not bridge:
        logger.error("Could not establish connection with any slave_id")
        return

    try:
        # Read basic data
        logger.info("Reading inverter registers...")
        
        device_status = await bridge.client.get(rn.DEVICE_STATUS, bridge.slave_id)
        input_power = await bridge.client.get(rn.INPUT_POWER, bridge.slave_id)
        active_power = await bridge.client.get(rn.ACTIVE_POWER, bridge.slave_id)
        power_factor = await bridge.client.get(rn.POWER_FACTOR, bridge.slave_id)
        
        print("\n=== Inverter Data ===")
        print(f"Status: {device_status.value}")
        print(f"Input power: {input_power.value}W")
        print(f"Active power: {active_power.value}W")
        print(f"Power factor: {power_factor.value}")

        # If there is a power meter, try to read its data
        if bridge.power_meter_type:
            logger.info("Reading Power Meter data...")
            print("\n=== Power Meter Data ===")
            
            power_meter_power = await bridge.client.get(rn.POWER_METER_ACTIVE_POWER, bridge.slave_id)
            power_meter_energy = await bridge.client.get(rn.GRID_ACCUMULATED_ENERGY, bridge.slave_id)
            
            print(f"Active power: {power_meter_power.value}W")
            print(f"Accumulated energy: {power_meter_energy.value}kWh")

    except HuaweiSolarException as e:
        logger.error(f"Error reading inverter data: {str(e)}")
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
    finally:
        if bridge:
            await bridge.stop()

if __name__ == "__main__":
    asyncio.run(main())

## Conclusion

This notebook demonstrated how to establish a direct connection to a Huawei Inverter and read basic data using the `huawei-solar` library. Ensure that no other clients are connected to the inverter's monitoring port while running this script.
